In [ ]:
import sys
import os
import torch
from tinytorch import *

In [ ]:
def testDraw(ValType):
  a = ValType(2.0, label='a', requires_grad=True)
  b = ValType(-3.00, label='b', requires_grad=True)
  c = ValType(10.0, label='c', requires_grad=True)
  e = a*b; 
  d = e + c                                                  ; assignLabel(d, 'd', requires_grad=True) 
  e = a*b                                                    ; assignLabel(e, 'e', requires_grad=True)                                    
  f = ValType(-2.0, label = 'f', requires_grad=True)
  L = d*f                                                    ; assignLabel(L, 'L', requires_grad=True)
  return L,f,e,d,c,b,a


In [ ]:
V1 = testDraw(Value)

In [ ]:
draw_dot(V1[0])

In [ ]:
V2 = testDraw(Tensor)

In [ ]:
draw_dot(V2[0])

In [ ]:
V1[0].backward()

In [ ]:
draw_dot(V1[0])

In [ ]:
V2[0].backward()

In [ ]:
draw_dot(V2[0])

In [ ]:
for a,b in zip(V1,V2):
  print(a.data, b.item(), a.grad, b.grad.item(), a.data == b.item() and a.grad == b.grad.item())

In [ ]:
for k in V2:
  print(k.label, k.op, k._prev)

# Now lets do the prior example of a very simple 3 layer perceptron

In [ ]:
import random
import numpy as np

class Layer:
  def __init__(s, id, nin, nout, ValType=torch.tensor, dtype=float):
    s.w = ValType([ random.uniform(-1,1) for k in range(nin*nout)], dtype=dtype).view(nout, nin); assignLabel(s.w, f'L{id}.w', requires_grad=True)
    s.b = ValType([random.uniform(-1,1) for k in range(nout)], dtype=dtype); assignLabel(s.b, requires_grad=True, label=f'L{id}.b')

  def __call__(s, x):
    outs = (s.w @ x + s.b).tanh() 
    return outs
  def parameters(s):
    return [s.w, s.b] 
    
class MLP:
  def __init__(s, nin, nouts, ValType=torch.tensor, dtype=float):
    """ nouts in a list of sizes for the individual layers """
    sz = [nin] + nouts
    s.layers = [Layer(i, sz[i], sz[i+1], dtype=dtype, ValType=ValType) for i in range(len(nouts))]
  def __call__(s,x):
    for layer in s.layers:
      x = layer(x)
    return x
  def parameters(s):
    return [ p for layer in s.layers for p in layer.parameters() ]
            


In [ ]:
xs = [
  [2.0, 3.0, -1.0],
  [3.0, -1.0, 0.5],
  [0.5, 1.0, 1.0],
  [1.0, 1.0, -1.0],
]
x = xs[0]
ys = [ 1.0, -1.0, -1.0, 1.0] # desired targets

In [ ]:
def buildMLP(nin, nouts, dtype=float, ValType=torch.tensor):
  n = MLP(nin, nouts, dtype=dtype, ValType=ValType)
  for i,k in enumerate(n.layers):
    print(f'Layer {i} params: {k.w.shape=}, {k.b.shape=}')
  print(f'number of parameters = {sum(map(lambda x: MUL(*x.shape, 1,1), [ x for x in n.parameters() ]))}')
  return n

In [ ]:
def doPass(nn, xs, ys, npass=20, ValType=torch.tensor, Zeros=torch.zeros, dtype=float, LR=0.05):
  assert len(xs) == len(ys), f'{len(xs)=} elements but {len(ys)=}'
  assert isinstance(nn, MLP), f'{type(MLP)} is not a {MLP}'
  YS = [ ValType(k,dtype=dtype) for k in ys ]
  #ebugPush(Tensor, Tensor.backward, Tensor.binaryOpHelper, Tensor.unaryOpHelper)
  losses = []
  for k in range(npass):
    # forward pass
    # produce len(xs) number of tensors
    ypred = [ nn(ValType(x,dtype=dtype)) for x in xs ]
    loss = sum((yout-ygt)**2 for ygt, yout in zip(YS, ypred))
    print(f'pass {k} {loss.item()}')
    # backward pass
    for p in nn.parameters():
      p.grad = Zeros(*p.shape, dtype=float)
    loss.backward()
    # update
    # note that in a real NN, the following manual adjustment would be handled automatically by the gradient descent function
    for p in nn.parameters():
      pD = -LR * p.grad
      # print(Shapes(p=p, MINUS_EQUAS='', pD=pD, LR=LR, pG=p.grad))
      p.requires_grad = False
      p += pD
      p.requires_grad = True
    losses.append(loss)
  return losses
  

In [ ]:
DebugPop()

In [ ]:
np.random.seed(1337)
random.seed(1337)

n1 = buildMLP(3, [4, 4, 1])

np.random.seed(1337)
random.seed(1337)

n2 = buildMLP(3, [4, 4, 1], ValType=Tensor)

a1 = n1(torch.tensor(xs[0], dtype=float))
a2 = n2(Tensor(xs[0],dtype=float))
assert abs(a1.item() - a2.item()) < 0.000001
##doPass(n1, xs, ys)

In [ ]:
# now lets draw the computation graph of a2
draw_dot(a2,Dir='LR')

In [ ]:
# now do it again
np.random.seed(1337)
random.seed(1337)

n1 = buildMLP(3, [4, 4, 1])
L1 = doPass(n1, xs, ys)

In [ ]:
#DebugPush(Tensor, Tensor.binaryOpHelper, Tensor.unaryOpHelper, Tensor.backward)
DebugPop()

In [ ]:
np.random.seed(1337)
random.seed(1337)

n2 = buildMLP(3, [4, 4, 1], ValType=Tensor)
L2 = doPass(n2, xs, ys, ValType=Tensor, Zeros=Tensor.zeros)

In [ ]:
assert len(L1) == len(L2)

In [ ]:
L1

In [ ]:
for i in range(len(L1)):
  print(L1[i].item(),  L2[i].item())
  assert abs(L1[i].item() - L2[i].item()) < 0.00001, f'Loss {i} is not identical {L1[i]} {L2[i]}'